## Introduction

Ce programme a pour but de tester la plausibilité de la prononciation d'un mot telle qu'indiquée sur le Wiktionnaire.

Pour ce faire, il parcours itérativement les différents sons (phonemes) d'un mot et essaie les différentes lettres (graphemes) pouvant être utilisées pour écrire le son (grâce à une table de correspondance similaire à https://fr.wiktionary.org/wiki/Annexe:Prononciation/français#Troisième_approche). A chaque essai, il compare ce qu'il obtient avec l'orthographe utilisée dans le Wiktionnaire. Lorsqu'il arrive au même résultat, la prononciation du mot est jugée plausible. A l'inverse, s'il n'arrive pas à transcrire la même orthographe que celle indiquée dans le Wiktionnaire, la prononciation est jugée suspecte.

Ce programme peut aussi compter les différentes lettres utilisées pour transcrire un son. Ainsi, s'il est executé sur tous les mots du Wiktionnaire, il peut servir à compter les probabilités de ces lettres pour transcrire un son donné.



## Données d'entrée

In [3]:
# importation des librairies tierces
import pandas as pd
from pathlib import Path

def add_temporary_words(df, mot, prononciation):
  df = df.append(dict(zip(['Wiki_page', 'Word', 'Pronunciation', 'Wiki_T'],
                          [mot, mot, prononciation, ''])), ignore_index=True)
  return df

# lecture des données d'entrée (ligne contenant un mot du dictionnaire
# ainsi que sa prononciation
def read_df():

  '''Read the input data (i.e. lines containing a Wiktionary word 
  and its pronunciation) '''
  filename = 'fr_wiktionary_extract.csv'
  filepath = Path(filename)
  url = 'https://fonétik.fr/'
  url_filename = url + filename

  print('downloading ', url_filename)
  # if file not found locally, then download it
  #if not filepath.exists():
  !wget -N -q {url_filename}

  df = pd.read_csv(filename, keep_default_na=False)

  # Ajout de certains prononciations de mots non-présentes dans les entrées
  # (manque la 2ème prononciation dans le csv)
  temporary_words  = [('xian', 'sjan'),
                      ('xhosa', 'kɔsa'),
                      ('argh', 'aʁ'),
                      ('bobsleigh', 'bɔbslɛ'),
                      ('bobsleighs', 'bɔbslɛ'),
                      ('os', 'o'),
                      ('résumpte', 'ʁezɔ̃t'),
                      ('taon', 'tɔ̃'),
                      ('taons', 'tɔ̃'),
                      ('exact', 'ɛɡza'),
                      ('exacts', 'ɛɡza'),
                       # should be in next version of csv
                      ('trapp', 'tʁap'),
                      ('trapps', 'tʁap'),
                      ('judd', 'ʒud'),
                      ('judds', 'ʒud'),
                      ('jodhs', 'jɔd'),
                      ('yachtsman', 'jɔtman'),
                      ('gonzs','ɡɔ̃z'),
                      ('hés','e'),
                      ('eins','ɛ̃'),
                      ('aveuent','avœ'),
                      ('hient','i'),
                      ('huent','y'),
                      ('hues','y'),
                      ('eaux-fortes','ofɔʁt'),
                      ('hauts-fonds','ofɔ̃'),
                      ('hoax','oks'),
                      ('curaçao','kyʁaso'),
                      ('curaçaos','kyʁaso'),
                      ('wisigoths','viziɡɔ'),
                      ('wisigoth','viziɡo'),
                      ('wisigoths','viziɡo'),
                      ('hon','ɔ̃'),
                      ('exempts','ɛɡzɑ̃'),
                      ('ah','a'),
                      ('s’', 's'),

  ]
  for temporary_word in temporary_words:
    df = add_temporary_words(df, temporary_word[0], temporary_word[1])

  return df

df = read_df()

downloading  https://fonétik.fr/fr_wiktionary_extract.csv


In [4]:
# Affichage du nombre d'échantillons en entrées
df.shape[0]

1422242

In [5]:
# Affichage de 5 derniers échantillons
df.tail(5)

,Wiki_page,Word,Pronunciation,Wiki_T
1422237,wisigoths,wisigoths,viziɡo,
1422238,hon,hon,ɔ̃,
1422239,exempts,exempts,ɛɡzɑ̃,
1422240,ah,ah,a,
1422241,s’,s’,s,


## Table de correspondance entre sons et lettres



In [0]:
# Table globale indiquant pour chaque phonème, les graphèmes possibles selon 
# la position du phonème dans le mot
phonemes2graphemes = {
    'm' : {
        'isolé' : {
            "m’" : {},
        },
        'début' : {
            'hm' : {},
            'm' : {},
            #'mh' : {},
        },
        'milieu' : {
            'm' : {},
            'me' : {},
            #'mh' : {},
            'mm' : {},
            'mme' : {},
        },
        'fin' : {
            'm' : {},
            'me' : {},
            'ment' : {},
            'mes' : {},
            'mm' : {},
            'mme' : {},
            'mment' : {},
            'mmes' : {},
            'mms' : {},
            'ms' : {},
        },
    },
    'v' : {
        'isolé' : {
        },
        'début' : {
            'v' : {},
            'w' : {},
        },
        'milieu' : {
            'bv' : {},
            'v' : {},
            've' : {}, ## écodéveloppements
            'w' : {},
            #'ʋ' : {},
        },
        'fin' : {
            'f' : {},
            'v' : {},
            've' : {},
            'vent' : {},
            'ves' : {},
            'vs' : {}, ## youvs
            'we' : {},
            'went' : {},
            'wes' : {},
        },
    },
    'f' : {
        'isolé' : {
        },
        'début' : {
            'f' : {},
            'ph' : {},
        },
        'milieu' : {
            'f' : {},
            'fe' : {},
            'ff' : {},
            'ffe' : {},
            'pf' : {},
            'ph' : {},
        },
        'fin' : {
            'f' : {},
            'fe' : {},
            'ff' : {}, ## apérispliff
            'fent' : {},
            'fes' : {},
            'ffe' : {},
            'ffent' : {},
            'ffes' : {},
            'fs' : {},
            'gh' : {},
            'pf' : {},
            'ph' : {},
            'phe' : {},
            'phent' : {},
            'phes' : {},
            'phs' : {},
            'v' : {},
            've' : {},
            'vs' : {},
        },
    },
      'b' : {
        'isolé' : {
        },
        'début' : {
            'b' : {},
            'bh' : {},
            'gb' : {}, ## gbagbiste (gabon)
        },
        'milieu' : {
            'b' : {},
            'be' : {},
            'bb' : {},
            'bbe' : {},
            'h' : {}, ## kahba
        },
        'fin' : {
            'b' : {},
            'bbe' : {},
            'bbent' : {},
            'bbes' : {},
            'be' : {},
            'bent' : {},
            'bes' : {},
            'bs' : {},
        },
    },
    'p' : {
        'isolé' : {
        },
        'début' : {
            'p' : {},
            #'ph' : {},
        },
        'milieu' : {
            'b' : {},
            'p' : {},
            'pe' : {},
            'pp' : {},
            'ps' : {}, ##corpsard
            'ppe' : {},
        },
        'fin' : {
            'p' : {},
            'pe' : {},
            'pent' : {},
            'pes' : {},
            'pp' : {},
            'ppe' : {},
            'ppent' : {},
            'ppes' : {},
            'pps' : {},
            'ps' : {},
        },
    },
    'd' : {
        'isolé' : {
            "d’" : {},
        },
        'début' : {
            'd' : {},
            'dh' : {},
            'j' : {}, ## for jaïnas
        },
        'milieu' : {
            'd' : {},
            'dd' : {},
            'de' : {},
            'dh' : {},
            'g' : {}, ## adagio, 
            'j' : {}, ## homejacker
            'z' : {},
        },
        'fin' : {
            'd' : {},
            'dd' : {},
            'dde' : {},
            'ddes' : {},
            'ddent' : {},
            'de' : {},
            'dent' : {},
            'des' : {},
            'dds' : {},
            'dh' : {},
            'dhs' : {},
            'ds' : {},
        },
    },
    't' : {
        'isolé' : {
            "t’" : {},
        },
        'début' : {
            'c' : {}, ## chatter
            't' : {},
            'th' : {},
        },
        'milieu' : {
            'c' : {}, ## rechatter
            'cht' : {},
            'chts' : {},
            'd' : {},
            'pt' : {},
            't' : {},
            'te' : {},
            'th' : {},
            'the' : {},
            'tt' : {},
            'tte' : {},
        },
        'fin' : {
            'cht' : {},
            'chts' : {},
            'd' : {},
            'dt' : {},
            'pt' : {},
            't' : {},
            'te' : {},
            'tent' : {},
            'tes' : {},
            'th' : {},
            'the' : {},
            'thent' : {},
            'thes' : {},
            'ths' : {},
            'ts' : {},
            'tt' : {},
            'tte' : {},
            'ttent' : {},
            'ttes' : {},
            'tts' : {},
        },
    },
    'n' : {
        'isolé' : {
            "n’" : {},
        },
        'début' : {
            'n' : {},
        },
        'milieu' : {
            'g' : {}, ## bignoler
            'hn' : {}, ## acehnais
            'mn' : {},
            'n' : {},
            'ñ' : {}, ## buñueliser
            'ne' : {},
            'nh' : {},
            'nn' : {},
            'nne' : {},
            'wn': {}, ## downloader
        },
        'fin' : {
            'mne' : {},
            'mnent' : {},
            'mnes' : {},
            'n' : {},
            'nh' : {},
            'nhs' : {},
            'ne' : {},
            'nent' : {},
            'nes' : {},
            'nn' : {},
            'nne' : {},
            'nnent' : {},
            'nnes' : {}, 
            'nns' : {},
            'ns' : {},
        },
    },
    'ɲ' : {
        'isolé' : {
        },
        'début' : {
            'gn' : {},
            'n' : {},
            'ñ' : {},
        },
        'milieu' : {
            'gn' : {},
            'gne' : {},
            'gni' : {}, ## torgnioler
            'ign' : {},
            'ñ' : {},
            'ni' : {}, ## orignal
            'nni' : {}, ## aiguillonniers
            'ny' : {},
        },
        'fin' : {
            'gne' : {},
            'gnent' : {},
            'gnes' : {},
            'igne' : {},
            'ny': {},  ## yidiny
            'nys': {}, ## yidinys
        },
    },
    'ŋ' : {
        'isolé' : {
        },
        'début' : {
            'n' : {},
        },
        'milieu' : {
            'n' : {},
            'ng' : {},
        },
        'fin' : {
            'g' : {}, ##hmong
            'ng' : {},
            'ngs' : {},
        },
    },
    'ɡ' : {
        'isolé' : {
        },
        'début' : {
            'g' : {},
            'gh' : {},
            'gu' : {},
        },
        'milieu' : {
            'c' : {},
            'g' : {},
            'gh' : {},
            'gg' : {},
            'gu' : {},
            'gue' : {},
        },
        'fin' : {
            'c' : {}, ## zinc
            'g' : {}, ## agreg
            'gge' : {},
            'ggent' : {},
            'gges' : {},
            #'ggs' : {}, ##briggs (not in Wix)
            'gs' : {},
            'gue' : {},
            'guent' : {},
            'gues' : {},
        },
    },
    'k' : {
        'isolé' : {
            "qu’" : {},
        },
        'début' : {
            'c' : {},
            'ch' : {},
            'k' : {},
            'kh' : {},
            'q' : {},
            'qu' : {}, ## questionneras
            'xh' : {},
        },
        'milieu' : {
            'c' : {},
            'cc' : {},
            'cch' : {},
            'ch' : {},
            'cg' : {}, ## brunschvicgien
            'ck' : {},
            #'ckh' : {},
            'cq' : {}, ## acquaintance 
            'cqu' : {},
            'cque' : {},
            'g' : {},
            'k' : {},
            'kh' : {},
            'kk' : {},
            'kw' : {}, ## atikamekwiser
            'q' : {},
            'qu' : {},
            'que' : {},
        },
        'fin' : {
            'c' : {},
            'ch' : {},
            'che' : {}, ## acéphalobrache
            'ches': {}, ## acritarches
            'chs' : {},
            'ck' : {},
            'cke' : {},
            'ckent' : {},
            'ckes' : {},
            'cks' : {},
            'cq' : {},
            'cque' : {},
            'cquent' : {},
            'cques' : {},
            'cs' : {},
            'g' : {},
            'k' : {},
            'ke' : {},
            'kent' : {},
            'kes' : {},
            'kh' : {},
            'khe' : {},
            'khs' : {},
            'khes' : {},
            'kke' : {},
            'kkent' : {},
            'kkes' : {},
            'ks' : {},
            'kw' : {}, ## atikamekw
            'kws' : {},
            'q' : {},
            'qs' : {},
            'que' : {},
            'quent' : {},
            'ques' : {},
        },
    },
    'ʒ' : {
        'isolé' : {
            "j’" : {},
        },
        'début' : {
            'g' : {},
            'ge' : {},
            'j' : {},
        },
        'milieu' : {
            'e' : {}, ## challengea
            'g' : {},
            'ge' : {},
            'gg' : {}, ## arpeggione
            'j' : {},
            'je' : {},
        },
        'fin' : {
            'g' : {},
            'ge' : {}, ## for voltage
            'gent' : {},
            'ges' : {},
            'j' : {},
            'je' : {},
            'jent' : {},
            'jes' : {},
            'js' : {},
        },
    },
    'ʃ' : {
        'isolé' : {
        },
        'début' : {
            'ch' : {},
            'che' : {},
            #!'h' : {}, ##chatter
            's' : {},
            'sch' : {},
            'sh' : {},
            'x' : {},
            'zh' : {},
        },
        'milieu' : {
            'c' : {}, ##acciaccatura
            'ch' : {},
            'che' : {},
            'chs' : {},
            'h' : {}, ##chatter
            's' : {},
            'sc' : {},
            'sch' : {},
            'sh' : {},
            'x' : {},
            'z' : {},##czamara
        },
        'fin' : {
            'ch' : {},
            'che' : {},
            'chent' : {},
            'ches' : {},
            'chs' : {},
            'sch' : {},
            'sche' : {},
            'schent' : {},
            'sches' : {},
            'schs' : {},
            'sh' : {},
            'she' : {},
            'shes' : {},
            'shent' : {},
            'shs' : {},
        },
    },
    'z' : {
        'isolé' : {
        },
        'début' : {
            'j' : {},
            'z' : {},
        },
        'milieu' : {
            's' : {},
            'se' : {},
            'si' : {},
            'x' : {},
            'z' : {},
            'zh' : {}, ##alzheimer
            'ze' : {},
            'zz' : {},
        },
        'fin' : {
            's' : {},
            'se' : {},
            'sent' : {},
            'ses' : {},
            'x' : {},
            'z' : {},
            'ze' : {},
            'zent' : {},
            'zes' : {},
            'zh' : {},
            'zs' : {},
            'zz' : {},
            'zze' : {},
            'zzent' : {},
            'zzes' : {},
        },
    },
    's' : {
       'isolé' : {
            "c’" : {},
            "s’" : {},
            "ç’" : {},
        },
        'début' : {
            'c' : {},
            'ç' : {},
            's' : {},
            'sc' : {},
            'th' : {},
            'x' : {},
        },
        'milieu' : {
            'c' : {},
            'cc' : {}, ##accenser
            'ç' : {},
            'ce' : {},
            's' : {},
            'sc' : {},
            'sç' : {},
            'se' : {},
            'ss' : {},
            'sse' : {},
            'sth' : {},
            't' : {},
            'x' : {},
            'z' : {},
        },
        'fin' : {
            'ce' : {},
            'cent' : {},
            'ces' : {},
            #'e' : {}, ##luxe (because of final e)
            's' : {},
            'sce' : {},
            'scent' : {},
            'sces' : {},
            'ss' : {},
            'se' : {},
            'sent' : {},
            'ses' : {},
            'sse' : {},
            'ssent' : {},
            'sses' : {},
            'th' : {},
            'ths' : {},
            'tz' : {},
            'x' : {},
            'z' : {},
        },
    },
    'ʁ' : {
        'isolé' : {
        },
        'début' : {
            #'hr' : {}, #Hrant
            'r' : {},
            'rh' : {},
            'rr' : {},
        },
        'milieu' : {
            'r' : {},
            'rc' : {}, ##antarctique
            're' : {},
            'rh' : {},
            'rr' : {},
            'rre' : {},
            'rrh' : {},
            'rrhe' : {},
            'rt': {}, ##courtbouillonnai
            'gh' : {}, # ????
        },
        'fin' : {
            'r' : {},
            'rc' : {},
            'rcs' : {},
            'rd' : {},
            'rds' : {},
            're' : {},
            'rent' : {},
            'res' : {},
            'rf' : {},
            'rfs' : {},
            'rg' : {},
            'rgh' : {},
            'rgs' : {},
            'rps' : {},
            'rr' : {},
            'rre' : {},
            'rrent' : {},
            'rres' : {},
            'rrhe' : {},
            'rrhent' : {},
            'rrhes' : {},
            'rrs' : {},
            'rs' : {},
            'rt' : {},
            'rts' : {},
            'gh' : {},
            'ghe' : {},
            'ghes' : {},
            'ghs' : {},
        },
    },
    'l' : {
        'isolé' : {
            "l’" : {},
        },
        'début' : {
            'l' : {},
            'll' : {},
        },
        'milieu' : {
            'e' : {}, ##googleliser
            'gl' :{}, ##imbroglio
            'l' : {},
            'le' : {},
            'li' : {}, ##aliya
            'lh' : {},
            'll' : {},
            'lle' : {},
        },
        'fin' : {
            'e' : {}, ##battle
            'es' : {}, ##crumbles
            'l' : {},
            'le' : {},
            'lent' : {},
            'les' : {},
            'lle' : {},
            'llent' : {},
            'lles' : {},
            'll' : {},
            'lls' : {},
            'ls' : {},
        },
    },
    'j' : {
        'isolé' : {
            'y' : {},
            'y’' : {},
        },
        'début' : {
            'hi' : {},
            'hy' : {},
            'i' : {},
            'ï' : {},
            'j' : {},
            'y' : {},
            'yi' : {}, ##yiable
        },
        'milieu' : {
            'e' : {}, ##newton
            'g': {}, ## designer
            'i' : {},
            'ï' : {},
            'ïi' : {}, ##abaïiez
            'ill' : {},
            'ille' : {},
            'illi' : {}, ## désembouteillions
            'j' : {},
            'gh' : {}, ##copyright
            'gui' : {},
            'hi' : {}, ##bahianiser
            'l' : {},
            'lh' : {}, ##abeilhanais
            'll' : {},
            'lle' : {},
            'lli' : {}, ##abrilliez
            'n' : {}, ##bignoler
            'u' : {}, ##barbecue
            'y' : {},
            'ye' : {},
            'yi' : {},
        },
        'fin' : {
            'e' : {}, ##bye
            'glie' : {},
            'glies' : {},
            'i' : {},
            'ï' : {},
            'ie' : {}, ##barbéie
            'ïe' : {},
            'ïes' : {},
            'ïent' : {}, ##abaïent
            'il' : {},
            'ille' : {},
            'illes' : {},
            'ils' : {},
            'illent' : {},
            'is' : {},
            'ïs' : {},
            'lle' : {},
            'llent' : {},
            'lles' : {},
            'y' : {},
            'ye' : {},
            'yent' : {},
            'yes' : {},
            'ys' : {},
        },
    },
    'w' : {
        'isolé' : {
        },
        'début' : {
            'ho' : {},
            'hou' : {},
            'ou' : {},
            'w' : {},
            'o' : {}, ## oiseau
        },
        'milieu' : {
            'hou' : {},
            'hu' : {},
            'o' : {},
            'oo' : {},
            'ou' : {},
            'oy' : {}, ## broyement
            'u' : {},
            'w' : {},
        },
        'fin' : {
            'w' : {},
        },
    },
    'ɥ' : {
        'isolé' : {
        },
        'début' : {
            'hu' : {},
        },
        'milieu' : {
            'hu' : {},
            'u' : {},
            'ü' : {},
            '’hu' : {}, ##aujourd’hui
        },
        'fin' : {
        },
    },
    'i' : {
        'isolé' : {
            'hi': {},
            'hie': {},
            'hient': {},
            'hies': {},
            'y': {},
        },
        'début' : {
            'hi' : {},
            'hie' : {},
            'hy' : {},
            'i' : {},
            'î' : {},
            'y' : {},
        },
        'milieu' : {
            'e' : {},
            'ea' : {},
            'ee' : {},
            'eo' : {}, ## peopliser
            'hi' : {},
            'hî' : {},
            'hy' : {}, ##réhydrater
            'i' : {},
            'î' : {},
            'ï' : {},
            'ie' : {},
            'ie' : {},
            'is' : {}, ## disjoncté
            'o': {}, ## businesswomen
            'u' : {},
            'ui' : {}, ## bodybuiledrons
            'y' : {},
            'yg' : {}, ## amygdalienne
            'ÿ' : {},            
        },
        'fin' : {
            'ee' : {}, ## banoffee
            'ees': {}, ## burpees
            'hi' : {},
            'hie' : {},
            'hies' : {},
            'his' : {},
            'hit' : {},
            'hît' : {},
            'hy' : {},
            'i' : {},
            'ï' : {},
            'î' : {}, ## barakî
            'id' : {},
            'ids' : {},
            'ict' : {},
            'icts' : {},
            'ie' : {},
            'ïe' : {},
            'ient' : {},
            'ies' : {},
            'ïes' : {},
            #'iest' : {}, #Saint-Priest
            'ii' : {},
            'il' : {},
            'ils' : {},
            'is' : {},
            'ïs' : {},
            'it' : {},
            'ît' : {},
            'its' : {},
            'ix' : {},
            'iz' : {},
            'y' : {},
            'ÿ' : {},
            'ye' : {},
            'yes' : {},
            'yie' : {},
            'yies' : {},
            'ys' : {},
            'yt' : {},
        },
    },
    'e' : {
        'isolé' : {
            'ai' : {},
            'eh' : {},
            'et' : {},
            'hé' : {},
            'hés' : {},
        },
        'début' : {
            'ae': {}, ## aedicule
            'ai' : {}, ## aigri
            'aî' : {}, ## aîné
            'æ' : {},
            'ay' : {},
            'e' : {},
            'é' : {},
            'ey' : {},
            'ha' : {},
            'he' : {},
            'hé' : {},
            'œ' : {},
            'oe' : {}, ## oedipianiser
        },
        'milieu' : {
            'a' : {},
            'ae' : {}, ## alaeforme
            'æ' : {},
            'ai' : {},
            'ay' : {}, ## abraysien
            'e' : {},
            'é' : {},
            'ê' : {}, ## bêcher
            'ë' : {}, ## aërométrie
            'ée' : {},
            'ei' : {}, ## baleinier
            'ey' : {},
            'hé' : {},
            'œ' : {},
            'oe' : {}, ## réoedipianiser
        },
        'fin' : {
            'ae' : {},
            'æ' : {},
            'aes' : {},
            'ai' : {},
            'ais' : {}, ## bandolais
            'ay' : {}, ## gray
            'ays' : {}, ## yoctograys
            'e' : {}, ## aguardiente
            'é' : {},
            'ë' : {},
            'ed' : {},
            'eds' : {},
            'eh' : {},
            'ehe' : {},
            'ehes' : {},
            'ehs' : {},
            'ee' : {},
            'é' : {},
            'ë' : {},
            'ër' : {}, ## argüer
            'ed' : {},
            'eds' : {},
            'eh' : {},
            'ehe' : {},
            'ehes' : {},
            'ehs' : {},
            'ee' : {},
            'et' : {}, ## nordet
            'ets' : {}, ## bouchets (err Wix?)
            'ée' : {},
            'éent' : {},
            'ées' : {},
            'ef' : {},
            'efs' : {},
            'er' : {},
            'ers' : {},
            'es' : {},
            'és' : {},
            'ës' : {},
            'ey' : {}, ## baloney
            'ex' : {},
            'ez' : {},
            'her' : {}, ## bréher
        },
    },
    'ɛ' : {
        'isolé' : {
            'aie' : {},
            'aient' : {},
            'aies' : {},
            'ais' : {},
            'ait' : {},
            'ayent' : {},
            'es' : {},
            'est' : {},
            'haie' : {},
            'haies' : {},
            'hais' : {},
            'hait' : {},
            'Haye' : {},
        },
        'début' : {
            'a' : {}, ## aces
            'ae' : {}, ## aerarium
            'æ' : {},
            'ai' : {},
            'aî' : {},
            'e' : {},       
            'é' : {},       
            'è' : {},
            'ê' : {},
            'ha' : {},
            'hay' : {},
            'he' : {},
            'hé' : {},
            'hê' : {},
            'œ' : {},
        },
        'milieu' : {
            'a' : {},
            'ä' : {}, ## bäckeofe
            'ae' : {},
            'æ' : {}, ## adhære
            'ai' : {},
            'aie' : {},
            'ay' : {},
            'aye' : {},
            'aî' : {},
            'e' : {},
            'ey' : {}, ## beylik
            'è' : {},
            'é' : {},
            'ê' : {},
            'ë' : {},
            'ea' : {},
            'ee' : {},
            'ei' : {},
            'es' : {},
            'hai' : {}, ## bréhaigne
            'he' : {},
            'hè' : {}, # bohème
            'i': {}, ## câliboire (Canada)
            'ue': {}, ## cuercir
            'œ' : {}, ## métœque
        },
        'fin' : {
            'ai' : {},
            'aict' : {},
            'aicts' : {},
            'aid' : {},
            'aids' : {},
            'aie' : {},
            'aient' : {},
            'aies' : {},
            'ais' : {},
            'ait' : {},
            'aît' : {},
            'aits' : {},
            'aix' : {},
            'ay' : {},
            'aye' : {},
            'ayes' : {},
            'ays' : {},
            'eah' : {}, ## yeah
            'ect' : {},
            'ects' : {},
            'eh' : {},
            'ehs' : {},
            'eigh' : {},
            'eighs' : {},
            'es' : {},
            'est' : {},
            'è' : {}, ## agapè
            'ès' : {},
            'et' : {},
            'ê' :  {}, ##corê
            'ês' :  {}, ##corês
            'êt' : {},
            'ets' : {},
            'êts' : {},
            'ey' : {},
            'eys' : {},
        },
    },
    'ɛ̃' : {
        'isolé' : {
            'aim' : {},
            'aims' : {},
            'ain' : {},
            'ains' : {},
            'ein' : {},
            'eins' : {},
            'haim' : {},
            'haims' : {},
            'hain' : {},
            'hains' : {},
            'hein' : {},
        },
        'début' : {
            'ain' : {},
            'ein' : {},
            'hin' : {},
            'im' : {},
            'in' : {},
        },
        'milieu' : {
            'ain' : {},
            'e' : {}, ## bienêtre
            'ein' : {},
            'en' : {}, ## bienvenue
            'im' : {},
            'in' : {},
            'ing' : {}, ## vingtuple
            'ïn': {}, ## coïnventiez
            'în' : {},
            'ym' : {},
            'yn' : {},
        },
        'fin' : {
            'aim' : {},
            'aims' : {},
            'ain' : {},
            'ainc' : {},
            'aincs' : {},
            'aing' : {},
            'aings' : {},
            'ains' : {},
            'aint' : {},
            'aints' : {},
            'ein' : {},
            'eins' : {},
            'eing' : {},
            'eings' : {},
            'eint' : {},
            'eints' : {},
            'en' : {},
            'ens' : {},
            'ent' : {},
            'in' : {},
            'inct' : {},
            'incts' : {},
            'ingt' : {},
            'ingts' : {},
            #'inq' : {}, #cinq-cent
            'ins' : {},
            'int' : {},
            'ints' : {}, ## adjoints
            'înt' : {},
            'ym' : {},
            'yms' : {},
            'yn' : {},
        },
    },    
    'œ̃' : {
        'isolé' : {
            'Hun' : {},
            'Huns' : {},
            'un' : {},
            'uns' : {},
        },
        'début' : {
            'hum' : {},
            'um' : {},
            #'un' : {}, #pas besoin dans unfollower 
        },
        'milieu' : {
            'un' : {},
        },
        'fin' : {
            'um' : {},
            'ums' : {},
            'un' : {},
            'uns' : {},
            'unt' : {},
            'unts' : {},
        },
    },
    'œ' : {
        'isolé' : {
        },
        'début' : {
            'heu' : {},
            'hœ' : {},
            'hu' : {},
            'œ' : {},
            'œu' : {},
            #'un' : {}, pas nécessaire dans unfollower
        },
        'milieu' : {
            'e' : {},
            'ea' : {}, ## yearling
            'eu' : {},
            'i' : {},
            'io' : {}, ## fashion
            'l' : {}, ## battle
            'œ' : {},
            'heu' : {}, ## aheurter
            'œu' : {},
            'ogl' : {},
            'oe' : {}, ## arroeiller
            'u' : {},
            'ue' : {},
        },
        'fin' : {
            'eu' : {},
            'euent' : {},
            #'eues' : {},
        },
    },
    'ə' : {
        'isolé' : {
        },
        'début' : {
        },
        'milieu' : {
            'ai' : {},
            'e' : {},
            'on' : {},
        },
        'fin' : {
            'e' : {},
        },
    },    
    'ø' : {
        'isolé' : {
            'euh' : {},
            'eux' : {},
            'œufs' : {},
        },
        'début' : {
            'e' : {}, #EDF
            'eu' : {},
            'heu' : {},
            'hœ' : {},
            'œ' : {},
        },
        'milieu' : {
            'e': {}, ## grelu
            'eu' : {},
            'eû' : {},
            'oe' : {},
            'œ' : {},
            'œu' : {}, ## bœufs
            'ö' : {}, ## börek
        },
        'fin' : {
            'eu' : {},
            'eud' : {}, ## alleud
            'eue' : {},
            'euent' : {},
            'eues' : {},
            'euf' : {}, ## éteuf
            'eufs' : {}, ## éteufs
            'euh' : {},
            'euhe' : {},
            'euhes' : {},
            'euhs' : {},
            'eur' : {},
            'eurs' : {},
            'eus' : {},
            'eut' : {},
            'euts' : {},
            'eux' : {},
            'œu' : {},
            'œud' : {},
            'œuds' : {},
            'œulx' : {},
            'œufs' : {}, ## bœufs
            'œux' : {},
        },
    },    
    'y' : {
        'isolé' : {
            'eu' : {},
            'eue' : {},
            'eues' : {},
            'eus' : {},
            'eut' : {},
            'eût' : {},
            'hue' : {},
            'huent' : {},
            'hues' : {},
        },
        'début' : {
            'eu' : {},
            'eû' : {},
            'hu' : {},
            'u' : {},
            'ü' : {},
        },
        'milieu' : {
            'eu' : {},
            'eü' : {},
            'hu' : {},
            'u' : {},
            'us' : {}, ## brusler
            'uu' : {}, ## buuz
            'ü' : {},
            'üe' : {}, ## argüerai
            'uë' : {}, ## arguëra
            'û' : {},
            'ue' : {},
        },
        'fin' : {
            'hu' : {},
            'hue' : {},
            'huent' : {},
            'hues' : {},
            'hus' : {},
            'hut' : {},
            'huts' : {},
            'u' : {},
            'ü' : {},
            'û' : {},
            'ue' : {},
            'uë' : {},
            'üe' : {},
            'uent' : {},
            'uënt' : {}, ## arguënt
            'üent' : {}, ## argüent
            'ues' : {},
            'uës' : {},
            'ües' : {},
            'ul' : {},
            'uls' : {},
            'upt' : {},
            'upts' : {},
            'us' : {},
            'ûs' : {},
            'ut' : {},
            'ût' : {},
            'uth' : {},
            'uths' : {},
            'uts' : {},
            'ûts' : {},
            'ux' : {},
            'uz' : {},
        },
    },    
    'u' : {
        'isolé' : {
            'hou' : {},
            'houe' : {},
            'houx' : {},
            'ou' : {},
            'où' : {},
            'ouh' : {},
        },
        'début' : {
            'aou' : {},
            'aoû' : {},
            'hou' : {},
            'hu' : {},
            'ou' : {},
            'oû' : {},
            'u' : {},
            'ü' : {},
        },
        'milieu' : {
            'aou' : {},
            'e' : {}, ## autointerviewa
            'hou' : {}, ## déhoussage
            'o' : {},
            'oe' : {}, ## broebèler (belgique)
            'oo' : {},
            'ou' : {},
            'oup' : {}, ## coupdebouliser
            'oû' : {},
            'oue' : {},
            'ow' : {},
            'u' : {},
            'ui' : {}, ## cruiser
            'ü' : {},
            'w' : {}, ## newton
        },
        'fin' : {
            'aoul' : {},
            'aouls' : {},
            'e': {}, ## barbecue
            'ew' : {},
            'ews' : {},
            'ou' : {},
            'où' : {}, ## ankoù
            'oub' : {},
            'oubs' : {},
            'ouc' : {},
            'oucs' : {},
            'oud' : {},
            'ouds' : {},
            'oue' : {},
            'ouent' : {},
            'ouh' : {},
            'oues' : {},
            'oug' : {},
            'ougs' : {},
            'ouls' : {}, ## pouls
            'oo' : {},
            'oos' : {},
            'oup' : {},
            'oups' : {},
            'ous' : {},
            'out' : {},
            'oût' : {},
            'outs' : {}, ## abouts
            'oûts' : {},
            'oux' : {},
            'u' : {},
            'us' : {},
        },
    },
    'o' : {
        'isolé' : {
            'au' : {},
            'aulx' : {},
            'aux' : {},
            'eau' : {},
            'eaux' : {},
            'haut' : {},
            'hauts' : {},
            'ho' : {},
            'ô' : {},
            'oh' : {},
            'os' : {},
        },
        'début' : {
            'au' : {},
            'aul' : {}, ## aulne
            'aux' : {}, ## auxdits
            'eau' : {},
            'eaux' : {},
            'ha' : {}, ## hall
            'hau' : {},
            'haut' : {},
            'hauts' : {},
            'heau' : {},
            'ho' : {},
            'hô' : {},
            'hoa' : {},
            'o' : {},
            'ô' : {},
            'oh' : {},
        },
        'milieu' : {
            'a' : {},
            'au' : {},
            'aû': {}, ## afaûtri
            'aw' : {}, ## crawlera
            'eo' : {}, ## yeoman
            'eau' : {}, ## beauvotte
            'hau' : {},
            'o' : {},
            'ô' : {},
            'ó' : {}, ## yóm
            'ō' : {}, ## yōkai
            'oa' : {},
            'oh' : {}, ## ohm
            'oo' : {},
        },
        'fin' : {
            'ao' : {},
            'aos' : {},
            'au' : {},
            'aud' : {},
            'auds' : {},
            'ault' : {},
            'aults' : {},
            'aulx' : {},
            'aus' : {},
            'aut' : {},
            'auts' : {},
            'auw' : {},
            'auws' : {},
            'aux' : {},
            'aw' : {},
            'aws' : {},
            'eau' : {},
            'eaus' : {},
            'eaux' : {},
            'hau' : {},
            'haut' : {},
            'hauts' : {},
            'ho' : {},
            'hos' : {},
            'o' : {},
            'ô' : {},
            'oc' : {},
            'ocs' : {},
            'oe' : {},
            'oh' : {},
            'oo' : {},
            'oos' : {},
            'ooz' : {},
            'op' : {},
            'ops' : {},
            'os' : {},
            'ôs' : {},
            'ot' : {},
            'ôt' : {},
            'oth' : {},
            'oths' : {},
            'ots' : {},
            'ôts' : {},
            'ow' : {},
            'ows' : {},
            'oz' : {},
        },
    },
    'ɔ' : {
        'isolé' : {
        },
        'début' : {
            'au' : {}, ## autocondionné
            'ho' : {},
            'o' : {},
        },
        'milieu' : {
            'a' : {},
            'ao' : {},
            'au' : {},
            'aw' : {}, ## yawl
            'ho' : {}, ## réhomologuerions
            'o' : {},
            'ö' : {}, ## coöpèrent
            'ô' : {}, ## yôghi
            'ō': {}, ## yōon
            'oa' : {},
            'oho' : {}, ## alcohol
            'oo' : {},
            'ou' : {},
            'ow' : {}, ## crowdsourcé
            'u' : {},
            'ü' : {},
            'œ' : {}, ## cœrcif
        },
        'fin' : {
            'o' : {}, ## agitato ###should not happen, err in Wix
            'os' : {}, ## allégrettos
            'ot' : {}, ## burot (err Wix?)
            'ots' : {}, ## burots (err Wix?)
        },
    },
    'ɔ̃' : {
        'isolé' : {
            'hon' : {},
            'on' : {},
            'ont' : {},
        },
        'début' : {
            'hon' : {},
            'hom' : {},
            'om' : {},
            'on' : {},
            'um' : {},
            'un' : {},
        },
        'milieu' : {
            'hon' : {},
            'om' : {},
            'omp' : {},
            'on' : {},
            'um' : {},
            'ump' : {},
            'un' : {},
            'unk' : {},
        },
        'fin' : {
            'aon' : {},
            'aons' : {},
            'om' : {},
            'omb' : {},
            'ombs' : {},
            'ompt' : {},
            'ompts' : {},
            'oms' : {},
            'on' : {},
            'onc' : {},
            'oncs' : {},
            'ond' : {},
            'onds' : {},
            'ong' : {},
            'ongs' : {},
            'ons' : {},
            'ons' : {},
            'ont' : {},
            'onts' : {},
        },
    },
    'ɑ̃' : {
        'isolé' : {
            'an' : {},
            'ans' : {},
            'en' : {},
            'han' : {},
        },
        'début' : {
            'e' : {}, # XM for enharmonie
            'am' : {},
            'an' : {},
            'em' : {},
            'en' : {},
            'ham' : {},
            'han' : {},
            'hen' : {},
        },
        'milieu' : {
            'a' : {}, ##bamba
            'aan' : {},
            'aën' : {},
            'am' : {},
            'amp' : {}, ##champlever
            'an' : {},
            'ang' : {}, ##bangkokien
            'e' : {}, ##renamourer
            'em' : {},
            'emp' : {},
            'en' : {},
            'han' : {},
            'hen' : {},
        },
        'fin' : {
            'aen' : {},
            'am' : {},
            'amp' : {},
            'amps' : {},
            'ams' : {},
            'an' : {},
            'anc' : {},
            'ancs' : {},
            'and' : {},
            'ands' : {},
            'ang' : {},
            'angs' : {},
            'ans' : {},
            'ant' : {},
            'ants' : {},
            'aon' : {},
            'aons' : {},
            'emps' : {},
            'empt' : {},
            'empts' : {},
            'end' : {},
            'ends' : {},
            'eng' : {},
            'engs' : {},
            'ens' : {},
            'ent' : {},
            'ents' : {},
            'han' : {},
            'hans' : {},
        },
    },
    'ɑ' : {
        'isolé' : {
        },
        'début' : {
            'a': {}, ##affrontation (probably an error in Wiktionary)
            'â' : {},
            'hâ' : {},
        },
        'milieu' : {
            'a' : {}, ##enfutailler
            'as' : {}, ##basfonds
            'â' : {},
            'aa' : {},
            'am' : {},
            'âh' : {}, ##brâhmane
            'i' : {}, ##reboiserait
            'î' : {}, ##boîter
        },
        'fin' : {
            'as' : {},
            'âs' : {},
            'ât' : {},
            'at' : {}, ##agnaca (Wix err)
            'ats' : {}, ##agnacas (Wix err)
            'âts' : {},
            'aw' : {}, ##bucksaw
            'az' : {},
        },
    },
    'a' : {
        'isolé' : {
            'a' : {},
            'à' : {},
            'ah' : {},
            'as' : {},
        },
        'début' : {
            'a' : {},
            'aa' : {},
            'ah' : {}, ##ahrimane
            'as' : {}, ##asnierois
            'ha' : {},
        },
        'milieu' : {
            'a' : {},
            'â' : {},
            'ã' : {}, ##baião
            'aa' : {},
            'ae' : {},
            'ah' : {}, ##bahreïni
            'ao' : {},
            'as' : {}, ##basbleuisa
            'at' : {}, ##étatsunianiser
            'e' : {},
            'ê' : {},
            'ha' : {},
            'i' : {},
            'î' : {},
            'ie' : {},
            'ig' : {},
            'o' : {}, ##burnouter
            'y' : {}, ##bye
        },
        'fin' : {
            'a' : {},
            'à' : {},
            'āʾ' : {}, ##yāʾ
            'aa' : {},
            'ac' : {},
            'ach' : {},
            'achs' : {},
            'acs' : {},
            'act' : {},
            'acts' : {},
            'ah' : {},
            'ahs' : {},
            'ap' : {},
            'aps' : {},
            'ars' : {},
            'as' : {},
            'at' : {},
            'ât' : {},
            'ats' : {},
            'âts' : {},
            'ax' : {},
            'ha': {}, ##baha
            'i' : {},
            'id' : {},
            'ids' : {},
            'ie' : {},
            'ient' : {},
            'ies' : {},
            'igt' : {},
            'igts' : {},
            'is' : {},
            'it' : {},
            'ît' : {},
            'its' : {},
            'îts' : {},
            'ix' : {},
            'ois' : {}, ##aderguois 
            'yes' : {},
        },
    },
    ## gngn de phonemes multiples pointant sur 1 seul et même graphème
    'aj' : {
        'isolé' : {
        },
        'début' : {
        },
        'milieu' : {
            'i' : {}, ##striker
            'y' : {}, ##flysurf
        },
        'fin' : {            
        },
    },   
    'dʒ' : {
        'isolé' : {
        },
        'début' : {
            'j' : {}, ##jaina
        },
        'milieu' : {
            'g' : {}, ##challenger
        },
        'fin' : {            
        },
    },       
    'ij' : {
        'isolé' : {
        },
        'début' : {
        },
        'milieu' : {
            'i' : {}, ##cypriote
        },
        'fin' : {            
        },
    },   
    'ks' : {
        'isolé' : {
        },
        'début' : {
            'x' : {}, ##xanthopie
        },
        'milieu' : {
            'x' : {}, ##excellent
        },
        'fin' : {
            'x': {}, ##solex
            'xe' : {}, ##luxe
            'xes' : {}, ##luxes
            'xent' : {}, ##fixent
        },
    },   
    'ɡz' : {
        'isolé' : {
        },
        'début' : {
            'x' : {} ##xénophobe
        },
        'milieu' : {
            'x' : {}, ##excellent
        },
        'fin' : {
        },
    },  
    'ɔɛ' : {
        'isolé' : {
        },
        'début' : {
        },
        'milieu' : {
            'œ' : {}, ##cœrcif
        },
        'fin' : {
        },
    },  
    'tʃ' : {
        'isolé' : {
        },
        'début' : {
            'ch' : {}, ##chatter
            'x' : {}, ##xavante
        },
        'milieu' : {
            'ch' : {}, ##rechatter
            'œ' : {}, ##acehnais
        },
        'fin' : {
        },
    },  
    'wa' : {
        'isolé' : {
        },
        'début' : {
        },
        'milieu' : {
            'o' : {}, ##ondoyés
        },
        'fin' : {
        },
    },   
    ## phonemes utilisés pour prononcer l'alphabet
    ## i.e. phonèmes multiples pointant sur un seul et même graphème
    'be' : {
        'isolé' : {
            'b': {}, 
        },
        'début' : {
            'b': {},
        },
        'milieu' : {
            'b' : {},
        },
        'fin' : {
            'b' : {},
        },
    }, 
    'se' : {
        'isolé' : {
            'c': {}, 
        },
        'début' : {
            'c': {},
        },
        'milieu' : {
            'c' : {},
        },
        'fin' : {
            'c' : {},
        },
    }, 
    'de' : {
        'isolé' : {
            'd': {}, 
        },
        'début' : {
            'd': {},
        },
        'milieu' : {
            'd' : {},
        },
        'fin' : {
            'd' : {},
        },
    }, 
    'ɛf' : {
        'isolé' : {
            'f': {}, 
        },
        'début' : {
            'f': {},
        },
        'milieu' : {
            'f' : {},
        },
        'fin' : {
            'f' : {},
        },
    }, 
    'ʒe' : {
        'isolé' : {
            'g': {}, 
        },
        'début' : {
            'g': {},
        },
        'milieu' : {
            'g' : {},
        },
        'fin' : {
            'g' : {},
        },
    }, 
    'aʃ' : {
        'isolé' : {
            'h': {}, 
        },
        'début' : {
            'h': {},
        },
        'milieu' : {
            'h': {},
        },
        'fin' : {
            'h': {},
        },
    }, 
    'ʒi' : {
        'isolé' : {
            'j': {}, 
        },
        'début' : {
            'j': {},
        },
        'milieu' : {
            'j' : {},
        },
        'fin' : {
            'j' : {},
        },
    }, 
    'ka' : {
        'isolé' : {
            'k': {}, 
        },
        'début' : {
            'k': {},
        },
        'milieu' : {
            'k' : {},
        },
        'fin' : {
            'k' : {},
        },
    }, 
    'ɛl' : {
        'isolé' : {
            'l': {},
        },
        'début' : {
            'l': {}, 
        },
        'milieu' : {
            'l': {},
        },
        'fin' : {
            'l': {},
        },
    }, 
    'ɛm' : {
        'isolé' : {
            'm': {}, 
        },
        'début' : {
            'm': {},
        },
        'milieu' : {
            'm': {},
        },
        'fin' : {
            'm': {},
        },
    }, 
    'ɛn' : {
        'isolé' : {
            'n': {}, 
        },
        'début' : {
            'n': {},
        },
        'milieu' : {
            'n': {},
        },
        'fin' : {
            'n': {},
        },
    }, 
    'pe' : {
        'isolé' : {
            'p': {}, 
        },
        'début' : {
            'p': {},
        },
        'milieu' : {
            'p': {},
        },
        'fin' : {
            'p': {},
        },
    }, 
    'ky' : {
        'isolé' : {
            'q': {}, 
        },
        'début' : {
            'q': {},
        },
        'milieu' : {
            'q': {},
        },
        'fin' : {
            'q': {},
        },
    }, 
    'ɛʁ' : {
        'isolé' : {
            'r': {}, 
        },
        'début' : {
            'r': {},
        },
        'milieu' : {
            'r': {},
        },
        'fin' : {
            'r': {},
        },
    }, 
    'ɛs' : {
        'isolé' : {
            's': {}, 
        },
        'début' : {
            's': {},
        },
        'milieu' : {
            's': {},
        },
        'fin' : {
            's': {},
        },
    }, 
    'te' : {
        'isolé' : {
            't': {},
        },
        'début' : {
            't': {}, 
        },
        'milieu' : {
            't': {},
        },
        'fin' : {
            't': {},
        },
    }, 
    've' : {
        'isolé' : {
            'v': {},
        },
        'début' : {
            'v': {}, 
        },
        'milieu' : {
            'v': {},
        },
        'fin' : {
            'v': {},
        },
    }, 
    'dubləve' : {
        'isolé' : {
            'w': {},
        },
        'début' : {
            'w': {}, 
        },
        'milieu' : {
            'w': {},
        },
        'fin' : {
            'w': {},
        },
    }, 
    'iks' : {
        'isolé' : {
            'x': {}, 
        },
        'début' : {
            'x': {}, 
        },
        'milieu' : {
            'x': {},
        },
        'fin' : {
            'x': {},
        },
    },    
    'iɡʁɛk' : {
        'isolé' : {
            'y': {}, 
        },
        'début' : {
            'y': {}, 
        },
        'milieu' : {
            'y': {},
        },
        'fin' : {
            'y': {},
        },
    }, 
    'zɛd' : {
        'isolé' : {
            'z': {},
        },
        'début' : {
            'z': {}, 
        },
        'milieu' : {
            'z': {},
        },
        'fin' : {
            'z': {},
        },
    },   
}

def add_keys():
  '''Ajoute des clés ('occurences', 'exemple_1', 'exemple_2', 'exemple_3')
   dans le dictionnaire de chaque grapheme de la table phonemes2graphemes.'''
  global phonemes2graphemes

  for phoneme in phonemes2graphemes.keys():
    for position in phonemes2graphemes[phoneme].keys():
      for grapheme in phonemes2graphemes[phoneme][position].keys():
        phonemes2graphemes[phoneme][position][grapheme]['occurences'] = 0
        phonemes2graphemes[phoneme][position][grapheme]['exemple_1'] = ''
        phonemes2graphemes[phoneme][position][grapheme]['exemple_2'] = ''
        phonemes2graphemes[phoneme][position][grapheme]['exemple_3'] = ''

# run add_keys
add_keys()

## Fonctions de comptage pour statistiques

In [0]:
# fonction de comptage qui incrémente d'une unité le compteur de correspondance
# entre un phonème donné et un graphème donné et qui enregistre en exemple 
# les trois premiers mots dans lequels la correspondance a été trouvé 
def increment_occurences(phoneme, position, grapheme, mot, unit_test=False):
  
  global phonemes2graphemes

  # ne pas continuer si l'appel provient d'un test unitaire
  if unit_test:
    return

  phonemes2graphemes[phoneme][position][grapheme]['occurences']  += 1
  if phonemes2graphemes[phoneme][position][grapheme]['exemple_1']  == '':
    phonemes2graphemes[phoneme][position][grapheme]['exemple_1'] = mot
  elif phonemes2graphemes[phoneme][position][grapheme]['exemple_2']  == '':
    phonemes2graphemes[phoneme][position][grapheme]['exemple_2'] = mot
  elif phonemes2graphemes[phoneme][position][grapheme]['exemple_3']  == '':
    phonemes2graphemes[phoneme][position][grapheme]['exemple_3'] = mot
  

In [8]:
# fonction de statistiques qui calcule pour pour chaque position (initiale,
# intermédiaire ou finale) le nombre d'occurences de chaque graphème ainsi
# que la probabilité de ce graphème.
# Les résultats sont insérés dans un DataFrame pandas et dans un fichier CSV.

def make_stats():
  global phonemes2graphemes

  table = []
  for phoneme in phonemes2graphemes:
    for position in ['isolé', 'début', 'milieu', 'fin']:
      for grapheme in phonemes2graphemes[phoneme][position].keys():
        occurences = phonemes2graphemes[phoneme][position][grapheme]['occurences']
        exemple_1 = phonemes2graphemes[phoneme][position][grapheme]['exemple_1']
        exemple_2 = phonemes2graphemes[phoneme][position][grapheme]['exemple_2']
        exemple_3 = phonemes2graphemes[phoneme][position][grapheme]['exemple_3']

        #print("phoneme=%s position=%s, grapheme=%s occurences=%d" % (phoneme, position, grapheme, occurences))
        row = { 'phonème': phoneme, 'position':position, 'graphème':grapheme, 'occurences':occurences, 'pourcentage':0.0,
               'exemple_1' : exemple_1, 'exemple_2' : exemple_2, 'exemple_3' : exemple_3, }
        table.append(row)
  dfp = pd.DataFrame.from_dict(table)

  for phoneme in dfp['phonème'].unique():
    df1 = dfp[dfp['phonème'] == phoneme]
    for position in df1['position'].unique():
      df2 = df1[df1['position'] == position]

      # calculate the sum of used graphemes within each phoneme-position category
      sum = 0
      for grapheme in df2['graphème'].unique():
        row_number = df2.loc[df2.graphème==grapheme].index[0]
        occurences = df2.at[row_number, 'occurences']
        sum += occurences

      # set the pourcentage of each graphemes for each phoneme-position category
      if sum != 0:
        for grapheme in df2['graphème'].unique():
          row_number = df2.loc[df2.graphème==grapheme].index[0]
          occurences = df2.at[row_number, 'occurences']
          pourcentage = int(occurences/sum*100)/100
          dfp.at[row_number, 'pourcentage'] = pourcentage

      dfp.to_csv("phonemes2graphemes.csv", index=False)

  return dfp

make_stats()

,phonème,position,graphème,occurences,pourcentage,exemple_1,exemple_2,exemple_3
0,m,isolé,m’,0,0.0,,,
1,m,début,hm,0,0.0,,,
2,m,début,m,0,0.0,,,
3,m,milieu,m,0,0.0,,,
4,m,milieu,me,0,0.0,,,
...,...,...,...,...,...,...,...,...
1394,iɡʁɛk,fin,y,0,0.0,,,
1395,zɛd,isolé,z,0,0.0,,,
1396,zɛd,début,z,0,0.0,,,
1397,zɛd,milieu,z,0,0.0,,,


## Fonction de test de plausibilité

In [0]:
# Fonction récursive tentant de retrouver tout ou partie des graphemes
# depuis tout ou partie des phonemes, à l'aide de la table de correspondance
# entre les phonemes et les graphemes.

def process_graphemes(mot, phonemes, graphemes, 
                      phoneme_index=0, grapheme_index=0, 
                      phonemes_hist='', graphemes_hist='', 
                      unit_test = False, verbose=False,):
  
  if verbose:
    print('')
    print('process_graphemes(phonemes=\\%s\\, graphemes=[[%s]], phoneme_index=%d, grapheme_index=%d)' 
            % (phonemes, graphemes, phoneme_index, grapheme_index))

  global phonemes2graphemes

  current_candidate_phonemes = []
  for phoneme in phonemes2graphemes:    
    if phonemes[phoneme_index:].startswith(phoneme):      
      current_candidate_phonemes.append(phoneme)
  if len(current_candidate_phonemes) == 0:
    return False, phonemes_hist, graphemes_hist
  if verbose:
    if len(current_candidate_phonemes) > 1:
      print("### candidate current phonemes=%", current_candidate_phonemes)

  nb_current_candidate_phonemes = 0

  for current_phoneme in current_candidate_phonemes:
    if verbose:
      print("trying candidate current phoneme=%s" % current_phoneme)
    nb_current_candidate_phonemes += 1
    next_phoneme = ''

    last_graphemes = graphemes[grapheme_index:]
    
    if verbose:
      print('process_graphemes(phonemes=\\%s\\, graphemes=[[%s]], phoneme_index=%d, grapheme_index=%d) current_phoneme:%s' 
            % (phonemes, graphemes, phoneme_index, grapheme_index, current_phoneme))
  
    if verbose:
      print('current_phoneme:%s' % current_phoneme)
    if phoneme_index == 0  and phonemes == current_phoneme:
      position = 'isolé'
    elif phoneme_index ==0 and phonemes[phoneme_index:].startswith(current_phoneme):
        position = 'début'
    else:
      if phoneme_index + len(current_phoneme) == len(phonemes):
        position = 'fin'
      else:
        position = 'milieu'

    if verbose:
      print('position:%s, current_phoneme:\\%s\\, last_graphemes:[[%s]]' % (position, current_phoneme, last_graphemes))

    try:
      len(phonemes2graphemes[current_phoneme][position])
    except:
      a=1
      if current_phoneme == '':
        if verbose:
          print('wrn: in [[%s]] : phoneme \\%s\\ does not exist !!!' % (graphemes, current_phoneme))
        return True, phonemes_hist, graphemes_hist
      else:
        print('err: in [[%s]] : phoneme \\%s\\ does not exist !!!' % (graphemes, current_phoneme))
        return False, phonemes_hist, graphemes_hist

    matching_graphemes_list = []

    for current_graphemes in phonemes2graphemes[current_phoneme][position]:
      
      if last_graphemes.startswith(current_graphemes):
        if verbose:
          print('phoneme \\%s\\ matching graphemes [[%s]] (last_graphemes:[[%s]])' % (current_phoneme, current_graphemes, last_graphemes))
        matching_graphemes_list.append(current_graphemes)

    if len(matching_graphemes_list) == 0:
      if verbose:
        print('KO0: phonemes=\\%s\\ does not match any grapheme !!!' % (phonemes))

      if nb_current_candidate_phonemes == len(current_candidate_phonemes):
        return False, phonemes_hist, graphemes_hist
      else:
        if verbose:
          print('move0 to next candidate phoneme')
        continue
      
    #print('current_phoneme=', current_phoneme)
    
    ### isolé ###
    if position == 'isolé':
      found = False
      for current_graphemes in matching_graphemes_list:
        if current_graphemes == graphemes:
          if verbose:
            print('ok: single phoneme=\\%s\\ matches full word [[%s]] ' % (phonemes, graphemes))
          phonemes_hist.append(current_phoneme)
          graphemes_hist.append(graphemes)
          #graphemes_hist2.append(graphemes)
          increment_occurences(current_phoneme, position, graphemes, mot, unit_test)
          return True, phonemes_hist, graphemes_hist#, graphemes_hist2
      if not found:
        #graphemes_hist2.append('?')
        if verbose:
          print('KO: phoneme=\\%s\\ matches no graphemes (in word=[[%s]]) !!!' % (phonemes, graphemes))
        return False, phonemes_hist, graphemes_hist#, graphemes_hist2

    ### début ###
    elif position == 'début':
      if len(matching_graphemes_list) == 0:
        if verbose:
          print('KO1: phonemes=\\%s\\ matches no grapheme (in word=[[%s]]) !!!' % (phonemes, graphemes))
        return False, phonemes_hist, graphemes_hist

      for current_graphemes in matching_graphemes_list:
        
        phonemes_hist.append(current_phoneme)
        graphemes_hist.append(current_graphemes)
        
        if verbose:
          print("trying1 current_phoneme \\%s\\ and current_graphemes [[%s]]" % (current_phoneme, current_graphemes))
        ret, phonemes_hist2a, graphemes_hist2a = process_graphemes(
            mot,
            phonemes, 
            graphemes,
            phoneme_index=phoneme_index+len(current_phoneme), 
            grapheme_index=grapheme_index+len(current_graphemes),
            phonemes_hist = phonemes_hist,
            graphemes_hist = graphemes_hist,
            unit_test = unit_test,
            verbose=verbose)
        
        phonemes_hist.pop()
        graphemes_hist.pop()
              
        if ret == True:
          if verbose:
            print('phonemes_hist2a:', phonemes_hist2a)
            print('graphemes_hist2a:', graphemes_hist2a)
            print("current_phoneme \\%s\\ and current_graphemes [[%s]] : worked ok" % (current_phoneme, current_graphemes))
          
          increment_occurences(current_phoneme, position, current_graphemes, mot, unit_test)
          return True, phonemes_hist2a, graphemes_hist2a
      
          graphemes_hist.pop()
          phonemes_hist.pop()

      # if here LOST
      if verbose:
          print('LOST1 for current_phoneme \\%s\\ in process_graphemes(phonemes=\\%s\\, graphemes=[[%s]], phoneme_index=%d, grapheme_index=%d)' 
                % (current_phoneme, phonemes, graphemes, phoneme_index, grapheme_index))

      if nb_current_candidate_phonemes == len(current_candidate_phonemes):
        return False, phonemes_hist2a, graphemes_hist2a
      else:
        if verbose:
          print('move1 to next candidate phoneme')

    ### milieu ###
    elif position == 'milieu':
          
      if len(matching_graphemes_list) == 0:
        if verbose:
          print('KO2: phoneme \\%s\\ matches NO grapheme (in word [[%s]])  !!!' % (graphemes, phonemes))
        phonemes_hist.pop()
        graphemes_hist.pop()
        return False, phonemes_hist, graphemes_hist

      for current_graphemes in matching_graphemes_list:
        
        phonemes_hist.append(current_phoneme)
        graphemes_hist.append(current_graphemes)

        if verbose:
          print("trying2 current_phoneme \\%s\\ and current_graphemes [[%s]]" % (current_phoneme, current_graphemes))

        ret, phonemes_hist2, graphemes_hist2 = process_graphemes(
            mot,
            phonemes, 
            graphemes,
            phoneme_index=phoneme_index+len(current_phoneme), 
            grapheme_index=grapheme_index+len(current_graphemes),
            phonemes_hist = phonemes_hist,
            graphemes_hist = graphemes_hist,
            unit_test = unit_test,
            verbose=verbose)

        if ret == True:
          if verbose:
            print('phonemes_hist2b:', phonemes_hist2)
            print('graphemes_hist2b:', graphemes_hist2)
            print("current_phoneme \\%s\\ and current_graphemes [[%s]] : worked ok" % (current_phoneme, current_graphemes))
          
          increment_occurences(current_phoneme, position, current_graphemes, mot, unit_test)
          return True, phonemes_hist2, graphemes_hist2
    
        phonemes_hist.pop()
        graphemes_hist.pop()
      
      # if here LOST
      if verbose:
          print('LOST2 for current_phoneme \\%s\\ in process_graphemes(phonemes=\\%s\\, graphemes=[[%s]], phoneme_index=%d, grapheme_index=%d)' 
                % (current_phoneme, phonemes, graphemes, phoneme_index, grapheme_index))
    
      if nb_current_candidate_phonemes == len(current_candidate_phonemes):
        return False, phonemes_hist2, graphemes_hist2
      else:
       
        if verbose:
          print('move2 to next candidate phoneme')
        continue

    ### END ###
    elif position == 'fin':

      if len(matching_graphemes_list) == 0:
        if verbose:
          print('KO3: phoneme=\\%s\\ matches NO grapheme (in word="%s")  !!' % (graphemes, phonemes))
        phonemes_hist.pop()
        graphemes_hist.pop()
        return False, phonemes_hist, graphemes_hist

      for current_graphemes in matching_graphemes_list:
        if current_graphemes == last_graphemes:
          phonemes_hist.append(current_phoneme)
          graphemes_hist.append(current_graphemes)
          #graphemes_hist2.append(current_graphemes)
          if verbose:
            print('OK3: "current_phoneme \\%s\\ and current_graphemes [[%s]] : ok, finally found graphemes=[[%s]] for phonemes=\\%s\\' % (current_phoneme, current_graphemes, graphemes, phonemes))
          increment_occurences(current_phoneme, position, current_graphemes, mot, unit_test)
          return True, phonemes_hist.copy(), graphemes_hist.copy()
      
      # if here not found
      # if here LOST
      if verbose:
          print('LOST3 for current_phoneme \\%s\\ in process_graphemes(phonemes=\\%s\\, graphemes=[[%s]], phoneme_index=%d, grapheme_index=%d)' 
                % (current_phoneme, phonemes, graphemes, phoneme_index, grapheme_index))
      
      if nb_current_candidate_phonemes == len(current_candidate_phonemes):
        return False, phonemes_hist, graphemes_hist 
      else:
        if verbose:
          print('move3 to next candidate phoneme')
      

## Tests

### Exemples

In [10]:
exemples = [
    ("momie", "mɔmi"),
    ("pookie", "puki"),

]

for exemple in exemples:
    
    mot = exemple[0]
    graphemes = exemple[0]
    phonemes = exemple[1]
    
    res = process_graphemes(mot, phonemes, graphemes, 
                            phoneme_index=0, grapheme_index=0, 
                            phonemes_hist=[], graphemes_hist=[], 
                            unit_test=True, verbose=False)
    print(res)

(True, ['m', 'ɔ', 'm', 'i'], ['m', 'o', 'm', 'ie'])
(True, ['p', 'u', 'k', 'i'], ['p', 'oo', 'k', 'ie'])


In [0]:
def check_word_pronunciation(mot, phonemes, graphemes, unit_test=False, verbose0=False, verbose1=False, verbose2=False):

    res, phonemes2, graphemes2  = process_graphemes(mot, phonemes, graphemes, phoneme_index=0, grapheme_index=0, phonemes_hist=[], graphemes_hist=[], unit_test=unit_test, verbose=verbose2)
    
    prononciation = ''.join(phonemes2)
    word = ''.join(graphemes2)
    prononciation_ = ','.join(phonemes2)
    word_ = ','.join(graphemes2)
    if prononciation != phonemes:
      if verbose0:
        print('[[%s]] \\%s\\ -> prononciation=\\%s\\ ' % (mot, phonemes, prononciation ))
      return False
    elif word != graphemes:
      if verbose0:
        print('[[%s]] != output graphemes=%s' % (mot, word))
      return True
    else:
      if verbose0:
        print('%s \\%s\\ -> %s \\%s\\' % (mot, prononciation, word_, prononciation_))
      return True

### Tests de référence

In [12]:
essais = [
    ("acéphalobrache", "asefalɔbʁak"),
    ("acquaintance","akwɛ̃tɑ̃s"),
    ("agoua","aɡwa"),
    ("bêchâmes","beʃam"),
    ("caouanne","kawan"),
    ("designer","dizajnœʁ"),
    ("désembouteillions","dezɑ̃butɛjɔ̃"),
    ("disjonctés","diʒɔ̃kte"),
    ("droitismes", "dʁwatism"),
    ("élégante", "eleɡɑ̃t"),
    ("enarrhas", "ɑ̃naʁa"),
    ("enfutailler", "ɑ̃fytɑje"),
    ("excaverai", "ɛkskavəʁe"),
    ("exemple", "ɛɡzɑ̃pl"),
    ("fashion","faʃœn"),
    ("khartoumisé","kaʁtumize"),
    ("hauts", "o"),
    ('hauts-fonds','o.fɔ̃'),
    ("html", "aʃteɛmɛl"), #acronyme
    ("intérêt", "ɛ̃teʁɛ"),
    ("intéressant","ɛ̃teʁɛsɑ̃"),
    ("jaïnas","dʒaina"),
    ("luxe","lyks"),
    ("momie","mɔmi"),
    ("oiseaux", "wazo"),
    ("ondoyés", "ɔ̃dwaje"),
    ("peopliser","piplize"),
    ("pookie", "puki"),
    ("rechaterais", "ʁətʃatəʁɛ"),
    ('solex', 'solɛks'),
    ('Paris', 'pa.ʁi'),
    ('VPN', 've.pe.ɛn'),
    
]

for essai in essais:
  check_word_pronunciation(essai[0], essai[1].replace('.',''), essai[0].lower().replace('-',''), unit_test=True, verbose0=True, verbose1=False, verbose2=False)

acéphalobrache \asefalɔbʁak\ -> a,c,é,ph,a,l,o,b,r,a,che \a,s,e,f,a,l,ɔ,b,ʁ,a,k\
acquaintance \akwɛ̃tɑ̃s\ -> a,cq,u,ain,t,an,ce \a,k,w,ɛ̃,t,ɑ̃,s\
agoua \aɡwa\ -> a,g,ou,a \a,ɡ,w,a\
bêchâmes \beʃam\ -> b,ê,ch,â,mes \b,e,ʃ,a,m\
caouanne \kawan\ -> c,a,ou,a,nne \k,a,w,a,n\
designer \dizajnœʁ\ -> d,e,s,i,g,n,e,r \d,i,z,a,j,n,œ,ʁ\
désembouteillions \dezɑ̃butɛjɔ̃\ -> d,é,s,em,b,ou,t,e,illi,ons \d,e,z,ɑ̃,b,u,t,ɛ,j,ɔ̃\
disjonctés \diʒɔ̃kte\ -> d,is,j,on,c,t,és \d,i,ʒ,ɔ̃,k,t,e\
droitismes \dʁwatism\ -> d,r,o,i,t,i,s,mes \d,ʁ,w,a,t,i,s,m\
élégante \eleɡɑ̃t\ -> é,l,é,g,an,te \e,l,e,ɡ,ɑ̃,t\
enarrhas \ɑ̃naʁa\ -> e,n,a,rrh,as \ɑ̃,n,a,ʁ,a\
enfutailler \ɑ̃fytɑje\ -> en,f,u,t,a,ill,er \ɑ̃,f,y,t,ɑ,j,e\
excaverai \ɛkskavəʁe\ -> e,x,c,a,v,e,r,ai \ɛ,ks,k,a,v,ə,ʁ,e\
exemple \ɛɡzɑ̃pl\ -> e,x,em,p,le \ɛ,ɡz,ɑ̃,p,l\
fashion \faʃœn\ -> f,a,sh,io,n \f,a,ʃ,œ,n\
khartoumisé \kaʁtumize\ -> k,ha,r,t,ou,m,i,s,é \k,a,ʁ,t,u,m,i,z,e\
hauts \o\ -> hauts \o\
hauts-fonds \ofɔ̃\ -> hauts,f,onds \o,f,ɔ̃\
html \aʃteɛmɛl\ -> h,

### Test en masse

In [13]:
df.shape

(1422242, 4)

In [14]:
# trier le dataframe (noms communs, acronymes, noms propres, puis restants)
# pour des exemples plus significatifs dans les statistiques
df_suffixes = df[df.Word.str.startswith('-')].copy()
df_autres0 = pd.concat([df, df_suffixes, df_suffixes]).drop_duplicates(keep=False)
df_nc = df_autres0[df_autres0.Word.str.islower()].copy()
df_autres = pd.concat([df, df_nc, df_nc]).drop_duplicates(keep=False)
df_np = df_autres[df_autres.Word.str.istitle()].copy()
df_autres2 = pd.concat([df_autres, df_np, df_np]).drop_duplicates(keep=False)
df_sigles = df_autres2[df_autres2.Word.str.isupper()].copy()
df_reste = pd.concat([df_autres2, df_sigles, df_sigles]).drop_duplicates(keep=False)
df2 = pd.concat([df_nc, df_sigles, df_np, df_suffixes, df_reste], ignore_index=True)
df2.shape

(1422042, 4)

In [15]:
#df2 = df
#df2=df2[df2.Word.str.istitle()] # ne conserver que les mots commençant par une majuscule puis minuscules
#df2=df2[df2.Word.str.islower()] # ne conserver que les mots entièrement en minuscules (i.e. noms communs)
#df2=df2[df2.Word.str.isupper()] # ne conserver que les mots entièrement en majuscules (i.e. acronymes)
#df2=df2[df2.Word.str.startswith('x')] # ne conserver que les mots noms communs commençant par ...

df2 = df2[~df2.Word.str.contains(' ')] # exclure les mots contenant un espace
#df2 = df2[~df2.Word.str.contains('-')] # exclure les mots contenant un tiret

df2 = df2[~df2.Word.str.contains('\.')] # exclure les mots contenant un point
df2 = df2[~df2.Word.str.contains('/')] # exclure les mots contenant un slash (e.g. copier/coller)

#df2 = df2[~df2.Word.str.contains('-')] # noms communs sans tiret dans le mot
df2.Word = df2.Word.str.replace('-','')
#df2.Word = df2.Word.str.replace("'",'')
#df2.Word = df2.Word.str.replace("’",'')


df2.Pronunciation = df2.Pronunciation.str.replace('(','')
df2.Pronunciation = df2.Pronunciation.str.replace(')','')
df2.Pronunciation = df2.Pronunciation.str.replace('‿','')
df2.shape

(1364777, 4)

In [16]:
nb_samples=0
nb_samples_ok=0
nb_samples_ko=0
nb_very_bad=0
verbose=False
for _, row in df2.iterrows():
    mot = row['Wiki_page']
    word = row['Word']
    pronunciation = row['Pronunciation']
      
    # if arrive here, it's ok
    nb_samples+=1
    try:
      is_ok = check_word_pronunciation(mot, pronunciation, word.lower(), False, False, False, False)
      if is_ok:
        nb_samples_ok+=1
      else:
        if verbose:
          if row['Wiki_T'] != 'flexions de verbe':
            print('word=%s \\\\%s\\\\'% (word, pronunciation))
        nb_samples_ko+=1
    except:
      nb_samples_ko+=1
      nb_very_bad+=1

if nb_samples > 0:
  print('nb_samples=%d, nb_samples_ok=%d, nb_samples_ko=%d, nb_very_bad=%d %%=%.2f' % (nb_samples, nb_samples_ok, nb_samples_ko, nb_very_bad, nb_samples_ok/nb_samples*100))

nb_samples=1364777, nb_samples_ok=1331570, nb_samples_ko=33207, nb_very_bad=0 %=97.57


###Tests unitaires

In [17]:
# Mot et prononciation à tester unitairement

#graphemes_phonemes='backstage \\bak.stɛdʒ\\'
graphemes_phonemes='brunch \\bʁœntʃ\\' 
mot_phonemes='hauts-fonds \\o.fɔ̃\\'
#graphemes_phonemes='aujourd’hui \\o.ʒuʁ.d‿ɥi\\'
strings = mot_phonemes.split(' ')
mot=strings[0]
graphemes=mot.lower() # pour noms propres ou acronymes
phonemes=strings[1]

# nettoyage des strings
graphemes = graphemes.replace('-','')
#graphemes = graphemes.replace("’",'')
phonemes = phonemes.replace(' ','')
phonemes = phonemes.replace('\\','')
phonemes = phonemes.replace('.','')
phonemes = phonemes.replace('‿','')
phonemes = phonemes.replace('(','')
phonemes = phonemes.replace(')','')

# test unitaire
check_word_pronunciation(mot, phonemes, graphemes, unit_test=True, verbose0=True, verbose1=True, verbose2=True)
#check_word_pronunciation(phonemes, graphemes, unit_test=True, verbose0=False, verbose1=False, verbose2=False)


process_graphemes(phonemes=\ofɔ̃\, graphemes=[[hautsfonds]], phoneme_index=0, grapheme_index=0)
trying candidate current phoneme=o
process_graphemes(phonemes=\ofɔ̃\, graphemes=[[hautsfonds]], phoneme_index=0, grapheme_index=0) current_phoneme:o
current_phoneme:o
position:début, current_phoneme:\o\, last_graphemes:[[hautsfonds]]
phoneme \o\ matching graphemes [[ha]] (last_graphemes:[[hautsfonds]])
phoneme \o\ matching graphemes [[hau]] (last_graphemes:[[hautsfonds]])
phoneme \o\ matching graphemes [[haut]] (last_graphemes:[[hautsfonds]])
phoneme \o\ matching graphemes [[hauts]] (last_graphemes:[[hautsfonds]])
trying1 current_phoneme \o\ and current_graphemes [[ha]]

process_graphemes(phonemes=\ofɔ̃\, graphemes=[[hautsfonds]], phoneme_index=1, grapheme_index=2)
trying candidate current phoneme=f
process_graphemes(phonemes=\ofɔ̃\, graphemes=[[hautsfonds]], phoneme_index=1, grapheme_index=2) current_phoneme:f
current_phoneme:f
position:milieu, current_phoneme:\f\, last_graphemes:[[utsfond

True

## Statistiques

In [0]:
# Réaliser les comptages tottax statistiques sur les correspondances 
# précédemment comptées
dfp = make_stats()

In [0]:
# Afficher les statistiques d'un phonème en particulier (ex: \s\)
phoneme = 'o'

In [20]:
dfp[dfp.phonème == phoneme][dfp.position == 'isolé']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,phonème,position,graphème,occurences,pourcentage,exemple_1,exemple_2,exemple_3
1012,o,isolé,au,1,0.06,au,,
1013,o,isolé,aulx,1,0.06,aulx,,
1014,o,isolé,aux,3,0.18,aux,-aux,-aux
1015,o,isolé,eau,3,0.18,eau,-eau,-eau
1016,o,isolé,eaux,1,0.06,eaux,,
1017,o,isolé,haut,1,0.06,haut,,
1018,o,isolé,hauts,1,0.06,hauts,,
1019,o,isolé,ho,1,0.06,ho,,
1020,o,isolé,ô,1,0.06,ô,,
1021,o,isolé,oh,2,0.12,oh,Oh,


In [21]:
dfp[dfp.phonème == phoneme][dfp.position == 'début']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,phonème,position,graphème,occurences,pourcentage,exemple_1,exemple_2,exemple_3
1023,o,début,au,35323,0.93,au-dehors,au-delà,au-dessous
1024,o,début,aul,14,0.00,aulnage,aulnages,aulnaie-frênaie
1025,o,début,aux,6,0.00,auxcrinier,auxcriniers,auxdites
1026,o,début,eau,50,0.00,eau-bénitier,eau-bénitiers,eau-de-vie
1027,o,début,eaux,1,0.00,eaux-vannes,,
1028,o,début,ha,2,0.00,hall,halls,
1029,o,début,hau,207,0.00,hauban,haubana,haubanage
1030,o,début,haut,65,0.00,haut-canadien,haut-commissaire,haut-commissariat
1031,o,début,hauts,12,0.00,hauts-commissaires,hauts-commissariats,hauts-de-chausse
1032,o,début,heau,3,0.00,heaume,heaumes,heaumier


In [22]:
dfp[dfp.phonème == phoneme][dfp.position == 'milieu']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,phonème,position,graphème,occurences,pourcentage,exemple_1,exemple_2,exemple_3
1039,o,milieu,a,143,0.00,ball-trap,balltrap,base-balls
1040,o,milieu,au,15931,0.11,abiauge,abiaugea,abiaugeai
1041,o,milieu,aû,4,0.00,afaûtri,afaûtris,jaûne
1042,o,milieu,aw,94,0.00,craw-craw,craw-craws,crawla
1043,o,milieu,eo,17,0.00,cache-oreille,cache-oreilles,esturgeonnière
1044,o,milieu,eau,1893,0.01,abeausi,abeausie,abeausies
1045,o,milieu,hau,1595,0.01,abat-chauvée,abat-chauvées,antihausse
1046,o,milieu,o,115218,0.82,abacos,abacost,abandogiciel
1047,o,milieu,ô,3921,0.02,acône,acônes,affrôla
1048,o,milieu,ó,3,0.00,ngaqómi,ngaqómis,yóm


In [23]:
dfp[dfp.phonème == phoneme][dfp.position == 'fin']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,phonème,position,graphème,occurences,pourcentage,exemple_1,exemple_2,exemple_3
1053,o,fin,ao,2,0.00,Curaçao,Guémené-Penfao,
1054,o,fin,aos,1,0.00,curaçaos,,
1055,o,fin,au,152,0.02,affutiau,affûtiau,agiau
1056,o,fin,aud,103,0.01,adriaud,artichaud,babaud
1057,o,fin,auds,68,0.00,adriauds,artichauds,attrape-nigauds
1058,o,fin,ault,29,0.00,boursault,cinsault,frinault
1059,o,fin,aults,6,0.00,boursaults,cinsaults,frinaults
1060,o,fin,aulx,3,0.00,faulx,topinambaulx,Paulx
1061,o,fin,aus,19,0.00,antitaus,berimbaus,burgaus
1062,o,fin,aut,53,0.00,assaut,aïaut,baricaut


## Bilan des correspondances non trouvées

* Normalement, aucune ligne ne devrait être affichée (si noms communs, noms propres et sigles analysés)

In [24]:
pd.set_option('display.max_rows', 100)
dfp[dfp.occurences == 0]

,phonème,position,graphème,occurences,pourcentage,exemple_1,exemple_2,exemple_3
462,j,isolé,y,0,0.0,,,
683,ɛ,isolé,Haye,0,0.0,,,
762,ɛ̃,isolé,Ain,0,0.0,,,
818,œ̃,isolé,Hun,0,0.0,,,
819,œ̃,isolé,Huns,0,0.0,,,
845,œ,milieu,ogl,0,0.0,,,
861,ø,début,hœ,0,0.0,,,
873,ø,fin,euent,0,0.0,,,
964,u,début,ü,0,0.0,,,
1172,ɑ̃,milieu,aën,0,0.0,,,


In [0]:
#from google.colab import files
#files.download('phonemes2graphemes.csv')